In [36]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_core.runnables import  RunnablePassthrough
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver
import hashlib
from langchain_core.prompts import SystemMessagePromptTemplate, PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate


from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

C:\Users\jocke\AppData\Local\Temp\ipykernel_2108\267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [10]:
loader = TextLoader(file_path="dummytext_good.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=280, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [4]:
llm = OllamaFunctions(model="llama3.1", temperature=0)


PROMPT = SystemMessagePromptTemplate.from_template("""
"I want you to generate a dataset of political arguments from debate platforms, each paired with 1 or more motivations. The dataset should have three columns: 'Argument,' 'Motivation Description,' and 'Max-Neef Need Category.' The Max-Neef categories are limited to: Subsistence, Protection, Affection, Understanding, Participation, Leisure, Creativity, Identity, Freedom.
Each argument should be a unique statement about a political topic (e.g., tax policy, healthcare, climate change), and the motivations must logically align with the argument.
Avoid duplicates and ensure coherence between the argument and its motivations. Below are five examples to guide you:

Argument: 'Tax policy should be expanded to support economic growth'
Motivation Description: 'Wants economic stability for all'
Max-Neef Need Category: Subsistence
                                                   
Argument: 'Climate change needs tighter restrictions to safeguard future generations'
Motivation Description: 'Wants to secure the future'
Max-Neef Need Category: Protection
Motivation Description: 'Desires collective action'
Max-Neef Need Category: Participation
                                                   
Argument: 'Healthcare policies should focus on advance social equity'
Motivation Description: 'Hopes to support the vulnerable'
Max-Neef Need Category: Affection
                                                   
Argument: 'Gun control needs tighter restrictions to protect public safety'
Motivation Description: 'Fears threats to safety'
Max-Neef Need Category: Protection
                                                   
Argument: 'Arts funding should prioritize foster cultural development'
Motivation Description: 'Enjoys cultural enrichment'
Max-Neef Need Category: Leisure
Motivation Description: 'Wants innovative solutions'
Max-Neef Need Category: Creativity

...
                                                                                 
Using these examples as a template, extract information from the input. Ensure each argument is distinct, politically relevant, and paired with motivations that make sense in context. Format you response with the 'Argument,' and 'Motivation Description,'"

""")

FINAL_TIP = HumanMessagePromptTemplate(
    prompt=PromptTemplate.from_template("""
The structure the nodes must obey are as follows:
    Argument: description
    Motivation: description; max neef category
The argument and motivation nodes must be related
Tip: Make sure to answer in the correct format and do not include any explanations.
Use the given format to extract information from the following input: {input}
""")
)

chat_prompt = ChatPromptTemplate.from_messages([PROMPT, FINAL_TIP])


allowed_relations = ["SUPPORTS", "AGAINST"]
allowed_nodes = ["Motivation", "Argument"]
#usar node_properties=["name"] caso necessário
#llm_transformer = LLMGraphTransformer(llm=llm, strict_mode=False, allowed_nodes=allowed_nodes, additional_instructions=additional_context)

#utilizar allowed_nodes gera um grafo mais sucinto, mas pode reduzir a complexidade
llm_transformer = LLMGraphTransformer(llm=llm, prompt=chat_prompt, allowed_nodes=allowed_nodes, node_properties=["description"], strict_mode=False)



C:\Users\jocke\AppData\Local\Temp\ipykernel_2108\1896400646.py:1: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1", temperature=0)


In [11]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph_documents[0]

GraphDocument(nodes=[], relationships=[], source=Document(metadata={'source': 'dummytext_good.txt'}, page_content="Title: A Resident's Appreciation: The New Airport Development\nUsername: Jane Smith\nFellow community members,"))

In [82]:
graph_documents_rascunho = graph_documents

In [77]:
graph_documents = graph_documents_rascunho

In [12]:
for doc in graph_documents:
    
        print(doc)



nodes=[] relationships=[] source=Document(metadata={'source': 'dummytext_good.txt'}, page_content="Title: A Resident's Appreciation: The New Airport Development\nUsername: Jane Smith\nFellow community members,")
nodes=[Node(id='Argument1', type='Argument', properties={'description': "I just wanted to share my thoughts on the new airport development in our area. As someone who has been following the progress closely, I'm thoroughly impressed with how the project is coming along."}), Node(id='Motivation1', type='Motivation', properties={'description': 'Supports local development', 'max_neef_category': 'Participation'})] relationships=[Relationship(source=Node(id='Argument1', type='Argument', properties={}), target=Node(id='Motivation1', type='Motivation', properties={}), type='SUPPORTS', properties={})] source=Document(metadata={'source': 'dummytext_good.txt'}, page_content="I just wanted to share my thoughts on the new airport development in our area. As someone who has been following\n

In [94]:
print(graph_documents[1].nodes)
print(graph_documents[1].relationships)

[Node(id='Argument_d1cf0e30', type='Argument', properties={'description': "I'm writing to express my strong opposition to the proposed international airport near our neighborhood."}), Node(id='Motivation_cc622f24', type='Motivation', properties={'description': 'Concerned about daily lives, environment, and quality of life', 'max_neef_category': 'Protection'})]
[Relationship(source=Node(id='Argument_d1cf0e30', type='Argument', properties={}), target=Node(id='Motivation_cc622f24', type='Motivation', properties={}), type='SUPPORTS', properties={})]


In [13]:

# Dicionário para mapear descrições para IDs únicos
description_to_id = {}

# Dicionário para mapear os IDs antigos para os novos (para atualizar relações)
old_to_new_id = {}

def generate_unique_id(description, type_label):
    """Gera um ID único baseado na descrição e tipo do nó."""
    if description not in description_to_id:
        content_hash = hashlib.md5(description.encode()).hexdigest()[:8]  # Hash curto
        unique_id = f"{type_label}_{content_hash}"  # Exemplo: "Argument_a1b2c3d4"
        description_to_id[description] = unique_id
    
    return description_to_id[description]

# Passo 1: Criar novos IDs para os nós e armazenar a conversão
for doc in graph_documents:
    for node in doc.nodes:
        description = node.properties.get("description", node.id)  # Usa a descrição se existir
        new_id = generate_unique_id(description, node.type)  # Gera o novo ID
        
        old_to_new_id[node.id] = new_id  # Guarda a conversão do ID antigo para o novo
        node.id = new_id  # Atualiza o ID do nó

    # Passo 2: Atualizar as relações para garantir que os IDs estão corretos
    for relation in doc.relationships:
        # Atualizar source e target para os novos IDs
        relation.source.id = old_to_new_id.get(relation.source.id, relation.source.id)
        relation.target.id = old_to_new_id.get(relation.target.id, relation.target.id)


In [96]:
print(old_to_new_id)

{'Argument1': 'Argument_2f12a7b2', 'Motivation1': 'Motivation_07590b22', 'Motivation2': 'Motivation_9f6d66b3'}


In [9]:
graph.add_graph_documents(graph_documents=graph_documents,  include_source=True)

In [10]:
# Atualizar a inserção no grafo para usar a label correta
for doc in graph_documents:
    for node in doc.nodes:
        entity_label = node.type.replace("-", "_").replace(" ", "_")  # Substituir hífen e espaços
        #print(node.properties)
        graph.query(
            f"""
            MERGE (n:{entity_label} {{id: $id, name: $name}})
            """,
            {"id": node.properties.get("description", node.id) , "name":node.id},
        )
    
    for relation in doc.relationships:
        relation_type = relation.type.replace(" ", "_")  # Garantir formato válido
        graph.query(
            f"""
            MATCH (a {{id: $source_id}}), (b {{id: $target_id}})
            MERGE (a)-[:{relation_type}]->(b)
            """,
            {"source_id": relation.source.properties.get("name"), "target_id": relation.target.properties.get("name")},
        )


In [ ]:
print(graph_documents)

AttributeError: 'list' object has no attribute 'nodes'

In [16]:
# adicionar nodes origem

# Criar ou obter o nó do tópico e conectar posts a ele
for doc in graph_documents:
    topic_name = doc.source.metadata.get("topic", "General Discussion")  # Obtém o nome do tópico
    topic_id = hashlib.md5(topic_name.encode()).hexdigest()[:8]  # Gera um ID único baseado no nome do tópico

    # Criar o nó do tópico de discussão no Neo4j
    graph.query(
        """
        MERGE (t:DiscussionTopic {id: $topic_id})
        SET t.name = $topic_name
        """,
        {"topic_id": topic_id, "topic_name": topic_name}
    )

    # Criar ou obter o nó do título do post
    post_title = doc.source.metadata.get("title", "Title: A Resident's Appreciation: The New Airport Development")
    post_id = hashlib.md5(post_title.encode()).hexdigest()[:8]

    graph.query(
        """
        MERGE (p:PostTitle {id: $post_id})
        SET p.title = $post_title
        """,
        {"post_id": post_id, "post_title": post_title}
    )

    # Conectar o post ao tópico de discussão
    graph.query(
        """
        MATCH (t:DiscussionTopic {id: $topic_id}), (p:PostTitle {id: $post_id})
        MERGE (p)-[:PART_OF]->(t)
        """,
        {"topic_id": topic_id, "post_id": post_id}
    )

    # Conectar argumentos ao post
    for node in doc.nodes:
        if "Argument" in node.type:
            graph.query(
                """
                MATCH (p:PostTitle {id: $post_id}), (n:Argument {id: $node_id})
                MERGE (n)-[:BELONGS_TO]->(p)
                """,
                {"post_id": post_id, "node_id": node.id}
            )

# Upload do grafo com argumentos e motivações
graph.add_graph_documents(graph_documents=graph_documents, include_source=False)



In [5]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [14]:
driver = GraphDatabase.driver(
        uri = os.getenv("NEO4J_URI"),
        auth = (os.getenv("NEO4J_USERNAME"),
                os.getenv("NEO4J_PASSWORD")))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [20]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting information from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = prompt | llm.with_structured_output(Entities)

In [21]:
entity_chain.invoke({"question":"What are the problems of building an airport?"})

Entities(names=['airport'])

In [22]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question":question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [23]:
print(graph_retriever("What are the problems of building an airport?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


In [24]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [25]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
chain.invoke(input="What are the problems of building an airport?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 116} for query: "CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
[#C455]  _: <CONNECTION> error: Failed to read from defunct connectio

'The proposed airport could have a significant impact on the community, including noise pollution, disruption to daily life, environmental concerns, and safety risks.'